In [36]:
# from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from glob import glob

In [37]:
# 종목 뉴스리스트 페이지별로 전체 html코드를 긁어온 후, 리스트 형식으로 반환
def getData(codeList, newsPage):
    pageList = []
    for code in codeList:
        for page in range(newsPage, newsPage+1):
            newsListPageUrl = 'https://finance.naver.com/item/news_news.naver?code='+str(code)+'&page='+str(page)+'&sm=title_entity_id.basic&clusterId='
            data = requests.get(newsListPageUrl)
            soup = BeautifulSoup(data.content.decode('euc-kr', 'replace'), 'html.parser')
            dataFilter = soup.find('tbody')
            pageList.append(dataFilter)
    return pageList

In [38]:
# 뉴스페이지별로 전체 html코드를 가공하여 각 뉴스기사들의 url주소를 만들어서, 리스트 형식으로 반환
def change(bs4Data, num):
    makeUrlList = []
    newsUrlList = []
    for bs4 in bs4Data:
        change = bs4.select('tbody > tr > td > a')
        for i in range(len(change)):
            if num <= 9:
                newsUrlList.append('https://finance.naver.com'+str(change[i])[21:65]+str(change[i])[69:83]+str(change[i])[87:99]+str(change[i])[103:110]+str(change[i])[114:138])
            elif num <= 99:
                newsUrlList.append('https://finance.naver.com'+str(change[i])[21:65]+str(change[i])[69:83]+str(change[i])[87:99]+str(change[i])[103:111]+str(change[i])[115:139])
            elif num <= 999:
                newsUrlList.append('https://finance.naver.com'+str(change[i])[21:65]+str(change[i])[69:83]+str(change[i])[87:99]+str(change[i])[103:112]+str(change[i])[116:140])
    return newsUrlList

In [39]:
def getNewsData(urlLists):
    newsDateList = []
    newsTitleList = []
    newsTextList = []
    for i in range(len(urlLists)):
        html = requests.get(urlLists[i])
        soup = BeautifulSoup(html.content.decode('euc-kr', 'replace'), 'html.parser')
        newsDateList.append(soup.find('span',{'class':'tah p11'}).get_text())
        newsTitleList.append(soup.find('strong', {'class':'c p15'}).get_text())
        newsText = soup.find('div', {'class':'scr01'})
        if newsText.find('div', {'class':'link_news'}) != None:
            newsText.find('div', {'class':'link_news'}).decompose()
        newsTextList.append(newsText.get_text()[1:-1])
    newsDf = pd.DataFrame({
        '날짜':newsDateList
        , '제목':newsTitleList
        , '내용':newsTextList
    })
    return newsDf

In [45]:
def filescan():
    fileDataList = []
    fileList = glob('./newsdata/*.csv')
    for i in range(len(fileList)):
        fileDataList.append(pd.read_csv('./newsdata/'+fileList[i][11:], encoding='utf-8'))
    hapData = pd.concat(fileDataList)
    hapData.drop_duplicates(['제목'], inplace=True)
    hapData.sort_values(['날짜'], inplace=True)
    hapData.reset_index(inplace=True)
    hapData.drop('index', axis=1, inplace=True)
    hapData.to_csv('./newsdata/하이닉스 뉴스 데이터.csv', encoding='utf-8', index=False)

In [151]:
def openFile():
    filtering = pd.read_csv('./newsdata/하이닉스 뉴스 데이터.csv', encoding='utf-8')
    for i in range(0, len(filtering)):
        filtering['날짜'][i] = filtering['날짜'][i].strip()
        filtering['내용'][i] = str(filtering['내용'][i]).replace('\t','')
    filtering.to_csv('./newsdata/하이닉스 뉴스 데이터.csv', encoding='utf-8', index=False)
    return pd.read_csv('./newsdata/하이닉스 뉴스 데이터.csv', encoding='utf-8')

In [152]:
stockCodeList = ['000660'] # 원하는 종목
stockNewsPage = 1        # 원하는 뉴스페이지 숫자 1 ~ 999페이지까지만 긁어오기 가능

for numberCount in range(1, stockNewsPage):
    bs4Data = getData(stockCodeList, numberCount)
    urlList = change(bs4Data, numberCount)
    getNewsDf = getNewsData(urlList)
    getNewsDf.to_csv('./newsdata/하이닉스 뉴스 데이터'+str(numberCount)+'.csv', encoding='utf-8', index=False) # 원하는 종목 코드에 맞는 종목 이름으로 꼭 바꿔서 사용하세요.

# 진행도는 여기서 확인할 수 없으니, ./newsdata/안에 있는 파일 생성되는 것을 확인하세요. -> 네이버 기사 페이지 숫자가 파일명에 붙어서 만들어집니다.
# 예) 하이닉스 뉴스 데이터77.csv -> 하이닉스 뉴스 77페이지 뉴스 리스트 다 긁어옴
# 네이버 봇이 자동으로 막는 문제인지 알 수 없으나, 30~50개 정도 파일을 만들어낼때마다 'Attribute error'가 뜨는데
# 하이닉스 뉴스 데이터93.csv 까지 파일이 만들어졌다가 에러가 떳다면
# for numberCount in range(93, stockNewsPage) <- 이 코드에서 네이버 뉴스 페이지 수 93을 
# 입력한채로 다시 실행하면 다시 만들기가 시작됩니다.
# 에러가 안뜰 수도 있음.
# 테스트 결과 하이닉스 총 344페이지 뉴스 페이지 전부 가져와지는 것을 확인함.

filescan()
openFile()

,날짜,제목,내용
0,2022.06.24 06:00,삼전·하이닉스 언제 반등?....미 반도체 지수 '촉각',기사내용 요약美필라델피아 반도체 지수 올 들어 33%↓ 하반기 업황도 '흐림'…주가...
1,2022.06.24 06:02,"SK하이닉스, 올해도 반도체 인재 육성 프로그램 실시",[비즈니스 플라자]SK하이닉스가 인재 채용이 필요한 우수 협력사와 취업 준비생을 연...
2,2022.06.24 08:09,"SK하이닉스, 매크로 악재로 메모리 수급 지연…목표가↓-NH",[이데일리 안혜신 기자] NH투자증권은 24일 SK하이닉스(000660)에 대해 ...
3,2022.06.24 08:53,"NH투자증권 “SK하이닉스, 메모리 수급 개선 지연...목표주가 14만원 하향”","글로벌 금리 인상, 유럽 전쟁, 중국 봉쇄 등으로 IT 제품 수요 부진메모리 공급 ..."
4,2022.06.24 18:38,"中, SK하이닉스 키파운드리 인수 승인",SK하이닉스 이천 로고. /연합뉴스 중국이 파운드리(반도체 위탁생산) ...
...,...,...,...
3275,2023.06.23 13:18,"SK하이닉스, 美 IT전시회서 차세대 메모리 솔루션 공개",SK하이닉스는 지난 20일(현지시간)부터 사흘간 미국 라스베이거스에서 열린 IT 전...
3276,2023.06.23 13:21,"델 이어 HPE까지…SK하이닉스, 美 HPE 전시회서 메모리 솔루션 소개",미국 ICT 기업 HPE 연례행사 HPE 디스커버 참가HBM3·DDR5·CXL·PI...
3277,2023.06.23 15:16,美 고객사들 만난 SK하이닉스…차세대 메모리 대거 공개,지난 20일(현지시간)부터 사흘간 미국 라스베이거스에서 열린 IT 전시회인 'HPE...
3278,2023.06.23 15:54,"삼성·SK하이닉스, 챗GPT용 메모리 시장 선점 경쟁…차세대 HBM 개발 ‘속도전’","삼성전자, 연내 ‘HBM3P’ 개발해 시제품 고객사에 전달”SK하이닉스에 뺏긴 시장..."
